In [2]:
#I created this file in this folder because, the necessary csv files are only present here. The same file "Analysis2.ipynb" is present
# ans this is only for the practice purpose for me !!!

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import webbrowser
import os

In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [4]:
#loading the data 
apps_df = pd.read_csv("Play Store Data.csv")
reviews = pd.read_csv("User Reviews.csv")

In [5]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [6]:
reviews.head()

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.00,0.533333
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.25,0.288462
2,10 Best Foods for You,NaN,NaN,NaN,NaN
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.40,0.875000
4,10 Best Foods for You,Best idea us,Positive,1.00,0.300000


In [7]:
#pd.read_csv -> To read csv file
#pd.read_excel -> for excel
#pd.read_sql -> for a sql file
#pd.read_json -> for a json file

In [8]:
#Real world data contains missings values, inconsistencies etc. It's important for us to clear these values.

#isnull() -> gives the missing vales..
#dropna() -> removes the missing values.
#fillna() -> fills the nan values
#check the pandas basics file for more info !!

In [9]:
#df.duplicated() : indentifies the duplicate rows
#df.drop_duplicates() : removes the duplicate rows

In [10]:
#Now lets clean the data in our playstore dataset..
apps_df = apps_df.dropna(subset=['Rating'])  #removes the rows with rating field nan , beacuse apps having no rating isn't useful for us. 
for col in apps_df.columns:   #looping through every column and replacing the values with the mode !!!
    apps_df[col].fillna(apps_df[col].mode()[0], inplace=True)
    
apps_df.drop_duplicates(inplace=True)
apps_df = apps_df[apps_df['Rating']<=5]  #filtering the data where any rating above 5 discarded

reviews.dropna(subset=['Translated_Review'], inplace=True)
 

C:\Users\HP\AppData\Local\Temp\ipykernel_65988\3351565794.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  apps_df[col].fillna(apps_df[col].mode()[0], inplace=True)


In [11]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs           object
Type               object
Price              object
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [12]:
#Covert the install columns into the int type by removing the comas, ect.!!!
apps_df['Installs'] = apps_df['Installs'].str.replace(',','').str.replace('+','').astype(int)
#replace all , + vlues to nothing and finally converting into the int

#Converting the price column to numeric removing the dollar sign
apps_df['Price'] = apps_df['Price'].str.replace('$', '').astype(float)

In [13]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews            object
Size               object
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [14]:
apps_df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


In [15]:
#merge the datasets, this allows to combine...
merged_df = pd.merge(apps_df, reviews, on='App', how='inner')
#Its basically combining the rows in both the tables on the App column !!!(inner means think of like a inner join !!)
#The non matching rows are excluded!!

In [16]:
merged_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity
0,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,A kid's excessive ads. The types ads allowed a...,Negative,-0.250000,1.000000
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,It bad >:(,Negative,-0.725000,0.833333
2,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,like,Neutral,0.000000,0.000000
3,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I love colors inspyering,Positive,0.500000,0.600000
4,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,I hate,Negative,-0.800000,0.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59119,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,1000000,Free,0.0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up,Nice broser slow browsing speed... make 8mbps ...,Positive,0.100000,0.492308
59120,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,1000000,Free,0.0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up,The thing I found missing simple bookmark draw...,Positive,0.225000,0.426786
59121,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,1000000,Free,0.0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up,Great Relief unwanted pop ups showing up. What...,Positive,0.650000,0.625000
59122,Firefox Focus: The privacy browser,COMMUNICATION,4.4,36981,4.0M,1000000,Free,0.0,Everyone,Communication,"July 6, 2018",5.2,5.0 and up,Hoped found new go-to; LOVE Firefox PC. Aside ...,Positive,0.345455,0.484848


Data Transformation

In [17]:
#This involves converting the ine form of data into the another
#Lets starts with the size ie., size of app ay be in MB, KB, GB and i want to standarised that
def convert_size(size):
    if 'M' in size:
        return float(size.replace('M', ''))
    elif 'K' in size:
        return float(size.replace('K', ''))/1024
    else :
        return np.nan
    
apps_df['Size'] = apps_df['Size'].apply(convert_size)

In [18]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device


In [19]:
#Logarithmic
apps_df['Log_Installs'] = np.log1p(apps_df['Installs'])
#This converts the values into their logarithmic values !!

In [20]:
apps_df['Reviews']=apps_df['Reviews'].astype(int)  #This makes the reviews column into the int type !!
apps_df['Log_Reviews'] = np.log1p(apps_df['Reviews'])


In [21]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,9.210440,5.075174
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,13.122365,6.875232
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,15.424949,11.379520
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,17.727534,12.281389
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,11.512935,6.875232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up,6.216606,2.079442
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,8.517393,3.663562
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,4.615121,1.609438
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device,6.908755,4.744932


In [22]:
apps_df.dtypes

App                object
Category           object
Rating            float64
Reviews             int32
Size              float64
Installs            int32
Type               object
Price             float64
Content Rating     object
Genres             object
Last Updated       object
Current Ver        object
Android Ver        object
Log_Installs      float64
Log_Reviews       float64
dtype: object

In [23]:
def rating_group(rating):
    if rating >=4 :
        return "Top Rated app"
    elif rating >=3:
        return "Above Average"
    elif rating >=2:
        return "Average"
    else :
        return "Below Average"
apps_df['Rating_Group'] = apps_df['Rating'].apply(rating_group)
        

In [24]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_Group
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,9.210440,5.075174,Top Rated app
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,13.122365,6.875232,Above Average
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,15.424949,11.379520,Top Rated app
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,17.727534,12.281389,Top Rated app
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,11.512935,6.875232,Top Rated app
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up,6.216606,2.079442,Top Rated app
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,8.517393,3.663562,Top Rated app
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,4.615121,1.609438,Top Rated app
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device,6.908755,4.744932,Top Rated app


In [25]:
#lets create a Revenue column
apps_df['Revenue'] = apps_df['Price']*apps_df['Installs']

In [26]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_Group,Revenue
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up,9.210440,5.075174,Top Rated app,0.0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up,13.122365,6.875232,Above Average,0.0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up,15.424949,11.379520,Top Rated app,0.0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up,17.727534,12.281389,Top Rated app,0.0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up,11.512935,6.875232,Top Rated app,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,"June 18, 2017",1.0.0,4.1 and up,6.216606,2.079442,Top Rated app,0.0
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,"July 25, 2017",1.48,4.1 and up,8.517393,3.663562,Top Rated app,0.0
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,"July 6, 2018",1.0,4.1 and up,4.615121,1.609438,Top Rated app,0.0
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,"January 19, 2015",Varies with device,Varies with device,6.908755,4.744932,Top Rated app,0.0


Sentiment Analysis Using NLP

In [27]:
#nltk library is used, it has a tool called SentimentAnalyser
# from nltk.sentiment.vader import SentimentIntensityAnalyzer (already imported above)
sia = SentimentIntensityAnalyzer()

#This gives 4 polarity scores, 1. Positive, 2. Negative, 3. Neutral, 4. Compound (between -1(very negative) to +1(means very positive), a normalised scores)

In [28]:
review = "This app is amazing! I love the new  features."
sentiment_score = sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.42, 'pos': 0.58, 'compound': 0.8516}


In [29]:
review = "This app is very bad! I hate the new  features."
sentiment_score = sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.535, 'neu': 0.465, 'pos': 0.0, 'compound': -0.8427}


In [30]:
review = "This app is okay."
sentiment_score = sia.polarity_scores(review)
print(sentiment_score)

{'neg': 0.0, 'neu': 0.612, 'pos': 0.388, 'compound': 0.2263}


In [31]:
reviews['Sentiment_Score'] = reviews['Translated_Review'].apply(lambda x : sia.polarity_scores(str(x))['compound'])#Adds the compound score into the table

In [32]:
reviews

,App,Translated_Review,Sentiment,Sentiment_Polarity,Sentiment_Subjectivity,Sentiment_Score
0,10 Best Foods for You,I like eat delicious food. That's I'm cooking ...,Positive,1.000000,0.533333,0.9531
1,10 Best Foods for You,This help eating healthy exercise regular basis,Positive,0.250000,0.288462,0.6597
3,10 Best Foods for You,Works great especially going grocery store,Positive,0.400000,0.875000,0.6249
4,10 Best Foods for You,Best idea us,Positive,1.000000,0.300000,0.6369
5,10 Best Foods for You,Best way,Positive,1.000000,0.300000,0.6369
...,...,...,...,...,...,...
64222,Housing-Real Estate & Property,Most ads older many agents ..not much owner po...,Positive,0.173333,0.486667,-0.6486
64223,Housing-Real Estate & Property,"If photos posted portal load, fit purpose. I'm...",Positive,0.225000,0.447222,0.7430
64226,Housing-Real Estate & Property,"Dumb app, I wanted post property rent give opt...",Negative,-0.287500,0.250000,-0.7269
64227,Housing-Real Estate & Property,I property business got link SMS happy perform...,Positive,0.800000,1.000000,0.7783


In [33]:
apps_df['Last Updated'] = pd.to_datetime(apps_df['Last Updated'], errors='coerce')

In [34]:
apps_df['Year'] = apps_df['Last Updated'].dt.year

In [35]:
apps_df

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver,Log_Installs,Log_Reviews,Rating_Group,Revenue,Year
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000,Free,0.0,Everyone,Art & Design,2018-01-07,1.0.0,4.0.3 and up,9.210440,5.075174,Top Rated app,0.0,2018
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000,Free,0.0,Everyone,Art & Design;Pretend Play,2018-01-15,2.0.0,4.0.3 and up,13.122365,6.875232,Above Average,0.0,2018
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000,Free,0.0,Everyone,Art & Design,2018-08-01,1.2.4,4.0.3 and up,15.424949,11.379520,Top Rated app,0.0,2018
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000,Free,0.0,Teen,Art & Design,2018-06-08,Varies with device,4.2 and up,17.727534,12.281389,Top Rated app,0.0,2018
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000,Free,0.0,Everyone,Art & Design;Creativity,2018-06-20,1.1,4.4 and up,11.512935,6.875232,Top Rated app,0.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,FR Calculator,FAMILY,4.0,7,2.6,500,Free,0.0,Everyone,Education,2017-06-18,1.0.0,4.1 and up,6.216606,2.079442,Top Rated app,0.0,2017
10836,Sya9a Maroc - FR,FAMILY,4.5,38,53.0,5000,Free,0.0,Everyone,Education,2017-07-25,1.48,4.1 and up,8.517393,3.663562,Top Rated app,0.0,2017
10837,Fr. Mike Schmitz Audio Teachings,FAMILY,5.0,4,3.6,100,Free,0.0,Everyone,Education,2018-07-06,1.0,4.1 and up,4.615121,1.609438,Top Rated app,0.0,2018
10839,The SCP Foundation DB fr nn5n,BOOKS_AND_REFERENCE,4.5,114,NaN,1000,Free,0.0,Mature 17+,Books & Reference,2015-01-19,Varies with device,Varies with device,6.908755,4.744932,Top Rated app,0.0,2015


Using the Plotly

In [36]:
#Plotly makes graphs interactive . It has bar, line charts, histgram etc. Plotly helps users to engage with the data
#Already imported the plotly !!!
html_files = "./"
if not os.path.exists(html_files):
    os.makedir(html_files)

In [40]:
plot_containers=""
def save_plot_as_html(fig, filename, insight):
    global plot_containers
    filepath = os.path.join(html_files, filename)
    html_content = pio.to_html(fig, full_html = False, include_plotlyjs='inline')
    plot_containers += f"""
    <div class = "plot-container" id="{filename}')" onclick="openPlot('{filename}')">
        <div class = "plot"> {html_content}</div> 
        <div class = "insights"> {insight}</div> 
    </div>
    """
    fig.write_html(filepath, full_html=False, include_plotlyjs='inline')
    

In [41]:
plot_width = 400
plot_height = 400
plot_bg_color='black'
text_color='white'
title_font={'size':16}
axis_font={'size':12}

In [42]:
# top 1 cateories in play store !!
#Figure 1
category_count = apps_df['Category'].value_counts().nlargest(10)   #we get the top 10 
fig1 = px.bar(
    x=category_count.index, 
    y=category_count.values,
    labels={'x':'Category', 'y':'Count'}, 
    title='Top Categories on PLay Store', 
    color = category_count.index,
    color_discrete_sequence=px.colors.sequential.Plasma, 
    width=400, height=300
)
#How the graph should be layouted
fig1.update_layout(
    plot_bgcolor='black',
    paper_bgcolor = 'black', 
    font_color = 'white', 
    title_font = {'size':16}, 
    xaxis=dict(title_font={'size':12}), 
    yaxis=dict(title_font={'size':12}), 
    margin=dict(l=10, r=10, t=30, b=10)
)

# fig1.update_traces(marker=dict(marker=dict(line=dict(color='white', width=1))))
save_plot_as_html(fig1, "Category Graph 1.html", "The top categories on the playstore are dominated by tools, entertainment and productivity apps")

In [44]:
#Figure 2 
#Lets use a pie chart, that tells the distribution between the free and paid apps

type_counts = apps_df['Type'].value_counts()   
fig2 = px.pie(
    values=type_counts.values,
    names=type_counts.index,
    title='App Type Distribution',     
    color_discrete_sequence=px.colors.sequential.RdBu,  #RdBu meansRed and Blue
    width=400, height=300
)
#How the graph should be layouted
fig2.update_layout(
    plot_bgcolor='black',
    paper_bgcolor = 'black', 
    font_color = 'white', 
    title_font = {'size':16},  
       
    margin=dict(l=10, r=10, t=30, b=10)
)

# fig1.update_traces(marker=dict(marker=dict(line=dict(color='white', width=1))))
save_plot_as_html(fig2, "Type Graph 2.html", "Most Apps on the playstore are free, indicating a strategy to attract users first and monetise through ads")

In [46]:
#Figure 3 
#Histogram
 
fig3 = px.histogram(
    apps_df,
    x='Rating',
    nbins=20,    
    title='Rating Distribution',     
    color_discrete_sequence=['#636EFA'],  #RdBu meansRed and Blue
    width=400, height=300
)
#How the graph should be layouted
fig3.update_layout(
    plot_bgcolor='black',
    paper_bgcolor = 'black', 
    font_color = 'white', 
    title_font = {'size':16},     
    margin=dict(l=10, r=10, t=30, b=10)
)

# fig3.update_traces(marker=dict(marker=dict(line=dict(color='white', width=1))))
save_plot_as_html(fig3, "Rating Graph 3.html", "Ratings are skewed towards higher values, suggesting that most apps are rated favorably by users")

In [47]:
#Figure 4
#Lets use the bar chart for sentiment analyzing

sentiment_counts = reviews['Sentiment_Score'].value_counts()   #we get the top 10 
fig4 = px.bar(
    x=sentiment_counts.index, 
    y=sentiment_counts.values,
    labels={'x':'Sentiment Score', 'y':'Count'}, 
    title='Sentiment Distribution', 
    color = sentiment_counts.index,
    color_discrete_sequence=px.colors.sequential.RdPu, 
    width=400, height=300
)
#How the graph should be layouted
fig4.update_layout(
    plot_bgcolor='black',
    paper_bgcolor = 'black', 
    font_color = 'white', 
    title_font = {'size':16}, 
    xaxis=dict(title_font={'size':12}), 
    yaxis=dict(title_font={'size':12}), 
    margin=dict(l=10, r=10, t=30, b=10)
)

# fig1.update_traces(marker=dict(marker=dict(line=dict(color='white', width=1))))
save_plot_as_html(fig4, "Sentiment Graph 4.html", "Sentiment in reviews show a mix of positive, negative feedback with a slight lean towards positive sentiments")